In [8]:
import requests
import json
from pprint import pprint
from datetime import datetime,time
from datetime import timedelta
import time as sleep
import sys
from configparser import ConfigParser
from requests.auth import HTTPBasicAuth
from urllib.parse import quote_plus

In [9]:
configur = ConfigParser() 
configur.read('config.ini')

# Telegram setup
bot_token = configur.get('telegram', 'bot_token')

# Telegram setup Pune
# Pune City
chatIdPuneCity = configur.get('telegram', 'PuneCity')

# Telegram setup Nashik Group
# Nashik City
chatIdNskCity = configur.get('telegram', 'NashikCity')

pincodeNashik = json.loads(configur.get('data', 'pincodeNashik'))
pincodePune = json.loads(configur.get('data', 'pincodePune'))
pincodes = [pincodeNashik] + [pincodePune]
pinCity = json.loads(configur.get('data', 'pinCity'))
global hitCount
hitCount = 0

In [10]:
def sendMsg(place,name,address,pin,vaccine,v_count,f_dose, s_dose, age, fee, date):
    try:
        if(place.upper() == "NashikCity".upper()):
            bot_chatID = chatIdNskCity
        elif(place.upper() == "PuneCity".upper()):
            bot_chatID = chatIdPuneCity

        msg = "ALERT! Slot available!\nName: " + str(name) + "\nAddress : " + str(address) + "\nVaccine : " + str(vaccine) + " : " + str(fee) + "\nAge : " + str(age) + "+" + "\n\nPincode : " + str(pin)+ "\nDate : " + str(date) + "\nTotal Slots : " + str(v_count) + "\n1st Dose slots : " + str(f_dose) + "\n2nd Dose slots : " + str(s_dose)
        msg = format(quote_plus(msg))
        send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&text=' + msg
        # print(send_text)
        response = requests.get(send_text)
        return response.json()
    except:
#         pass
        print(sys.exc_info())


In [11]:
def checkSlotsByPin(pincodes):
    global hitCount
#     print(hitCount)
    i = 0
    for pincode in pincodes:
        for pin in pincode:
            try:
                print(pin)
                today = datetime.now().date() + timedelta(1)
#                 print(today)
                today = today.strftime('%d-%m-%Y');
                url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByPin?pincode="+str(pin)+"&date="+str(today)

                headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
                response = requests.get(url, headers=headers)
#                 print(response)
#                 print(hitCount)
                hitCount += 1
                response = response.content.decode()
#                 print(response)

                jsonData = json.loads(response)
                jsonData = jsonData['centers']
#                 pprint(jsonData)
                for center in jsonData:
                    centerName = center['name']
                    address = center['address']
                    pin = center['pincode']
                    fee = center['fee_type']
                    for sessions in center['sessions']:
#                         print(sessions)
                        if (today == sessions['date']):
                            if(sessions['available_capacity'] > 0):
                                if center['center_id'] in entry_list:
                                    pass
                                else:
                                    print("Slot Available!")
                                    entry_list.append(center['center_id'])
                                    sendMsg(pinCity[i],centerName,address,pin,sessions['vaccine'],sessions['available_capacity'], sessions['available_capacity_dose1'], sessions['available_capacity_dose2'], sessions['min_age_limit'], fee, sessions['date'])
                            print(pinCity[i],centerName,address,pin,sessions['vaccine'],sessions['available_capacity'], sessions['available_capacity_dose1'], sessions['available_capacity_dose2'], sessions['min_age_limit'], fee, sessions['date'])

            except:
                pass
            sleep.sleep(2)
#                 print(sys.exc_info())
#         print(i)
        i += 1


In [ ]:
done = False
i = 1
entry_list = []
while not done:
    
    hitCount = 0
    print(hitCount)
    now = datetime.now().time()
    
    #By Pin
    checkSlotsByPin(pincodes)
    
    print("loop : " + str(i) + " Time : " + str(now))
    i += 1
    print(entry_list)
    sleep.sleep(300)
    if(i%3 == 0):
        #list cleared
        entry_list = []
    print(hitCount)


0
422001
{"centers":[]}
[]
422002
{"centers":[]}
[]
422003
{"centers":[{"center_id":590285,"name":"1 Apollo Hospital","address":"Plot No. 1 Swaminarayan Nagar New Adgaon Naka Panchavati Near Lunge Mangal Karyalay Nashik Maharashtra","state_name":"Maharashtra","district_name":"Nashik","block_name":"Nashik Corporation","pincode":422003,"lat":0,"long":23,"from":"12:30:00","to":"16:30:00","fee_type":"Paid","sessions":[{"session_id":"a2c4e406-8db8-4e97-9492-057b2c5ea622","date":"14-06-2021","available_capacity":3,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["12:30PM-01:30PM","01:30PM-02:30PM","02:30PM-03:30PM","03:30PM-04:30PM"],"available_capacity_dose1":0,"available_capacity_dose2":3},{"session_id":"2ae7cb17-19ee-415a-b51f-be77ecfee1e1","date":"14-06-2021","available_capacity":6,"min_age_limit":18,"vaccine":"COVISHIELD","slots":["10:30AM-12:30PM"],"available_capacity_dose1":0,"available_capacity_dose2":6},{"session_id":"f6c848cf-a8e6-4ea4-80b0-ed72c68bfd5b","date":"14-06-2021","avai

422104
{"centers":[]}
[]
422105
{"centers":[]}
[]
422112
{"centers":[]}
[]
422113
{"centers":[]}
[]
422201
{"centers":[]}
[]
422202
{"centers":[]}
[]
422203
{"centers":[{"center_id":58015,"name":"Sc Girnare Tal Nashik","address":"Girnare SC","state_name":"Maharashtra","district_name":"Nashik","block_name":"Nashik","pincode":422203,"lat":20,"long":73,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"ae6a1425-c72d-4f46-a4e1-141c867b2f9e","date":"14-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"],"available_capacity_dose1":0,"available_capacity_dose2":0}]}]}
[{'address': 'Girnare SC',
  'block_name': 'Nashik',
  'center_id': 58015,
  'district_name': 'Nashik',
  'fee_type': 'Free',
  'from': '09:00:00',
  'lat': 20,
  'long': 73,
  'name': 'Sc Girnare Tal Nashik',
  'pincode': 422203,
  'sessions': [{'available_capacity': 0,
                'available_ca

422209
{"centers":[]}
[]
422210
{"centers":[]}
[]
422211
{"centers":[{"center_id":55585,"name":"Barhe PHC","address":"Barhe PHC","state_name":"Maharashtra","district_name":"Nashik","block_name":"Surgana","pincode":422211,"lat":20,"long":73,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"a5d6c0d7-8910-452f-95b4-66efa7f80188","date":"14-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"],"available_capacity_dose1":0,"available_capacity_dose2":0},{"session_id":"145a2f1d-21ae-4f37-bae7-c84c21194c95","date":"15-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"],"available_capacity_dose1":0,"available_capacity_dose2":0},{"session_id":"7c529754-fcd9-44be-9553-376eb0786ee0","date":"16-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD",

411001
{"centers":[{"center_id":736344,"name":"Usha Nursing Home","address":"Usha Nursing Home 149 M G Road Pune","state_name":"Maharashtra","district_name":"Pune","block_name":"Haveli","pincode":411001,"lat":18,"long":73,"from":"12:30:00","to":"16:30:00","fee_type":"Paid","sessions":[{"session_id":"c25e7338-c988-40e9-a025-e91d8af3f3f4","date":"14-06-2021","available_capacity":0,"min_age_limit":18,"vaccine":"COVISHIELD","slots":["12:30PM-01:30PM","01:30PM-02:30PM","02:30PM-03:30PM","03:30PM-04:30PM"],"available_capacity_dose1":0,"available_capacity_dose2":0},{"session_id":"8864b43d-c3b4-4d0f-9aa2-14dfb5c6c50e","date":"14-06-2021","available_capacity":5,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["12:30PM-01:30PM","01:30PM-02:30PM","02:30PM-03:30PM","03:30PM-04:30PM"],"available_capacity_dose1":0,"available_capacity_dose2":5}],"vaccine_fees":[{"vaccine":"COVISHIELD","fee":"780"}]}]}
[{'address': 'Usha Nursing Home 149 M G Road Pune',
  'block_name': 'Haveli',
  'center_id': 73634

PuneCity Joshi Hospital Pune PMC Kamala Nehru Park Pune 411004 COVISHIELD 9 0 9 45 Paid 14-06-2021
PuneCity Joshi Hospital Pune PMC Kamala Nehru Park Pune 411004 COVISHIELD 0 0 0 18 Paid 14-06-2021
Slot Available!
PuneCity BMCC-DEENANATH HOSPITAL PVT BMCC COLLEGE SHIVAJINAGAR PUNE 411004 COVISHIELD 824 824 0 18 Paid 14-06-2021
PuneCity SAHYADRI SUPER SPECIALITY PMC KARVE ROAD DECCAN PUNE 411004 COVISHIELD 0 0 0 45 Paid 14-06-2021
PuneCity SAHYADRI SUPER SPECIALITY PMC KARVE ROAD DECCAN PUNE 411004 COVISHIELD 0 0 0 18 Paid 14-06-2021
411005
{"centers":[]}
[]
411006
{"centers":[]}
[]
411007
{"centers":[{"center_id":611307,"name":"SHASHWAT HOSPITAL AUND","address":"DAV PUBLIC SCHOOL AUNDH PUNE","state_name":"Maharashtra","district_name":"Pune","block_name":"Haveli","pincode":411007,"lat":18,"long":73,"from":"09:00:00","to":"16:00:00","fee_type":"Paid","sessions":[{"session_id":"d6014323-1755-48bd-9a7c-8dfb90881739","date":"16-06-2021","available_capacity":0,"min_age_limit":18,"vaccine":"C

...

411058
{"centers":[{"center_id":706828,"name":"MAI MANGESHKAR HOSPI WARJE PMC","address":"117 NEAR BAIF MUMBAI BANGLORE HIGHWAY WARJE","state_name":"Maharashtra","district_name":"Pune","block_name":"Haveli","pincode":411058,"lat":18,"long":73,"from":"10:00:00","to":"11:00:00","fee_type":"Paid","sessions":[{"session_id":"4064b069-a7c5-4c57-b113-17e456c7ce73","date":"14-06-2021","available_capacity":0,"min_age_limit":18,"vaccine":"COVISHIELD","slots":["10:00AM-11:00AM"],"available_capacity_dose1":0,"available_capacity_dose2":0}],"vaccine_fees":[{"vaccine":"COVISHIELD","fee":"780"}]}]}
[{'address': '117 NEAR BAIF MUMBAI BANGLORE HIGHWAY WARJE',
  'block_name': 'Haveli',
  'center_id': 706828,
  'district_name': 'Pune',
  'fee_type': 'Paid',
  'from': '10:00:00',
  'lat': 18,
  'long': 73,
  'name': 'MAI MANGESHKAR HOSPI WARJE PMC',
  'pincode': 411058,
  'sessions': [{'available_capacity': 0,
                'available_capacity_dose1': 0,
                'available_capacity_dose2': 0,
   

422104
{"centers":[]}
[]
422105
{"centers":[]}
[]
422112
{"centers":[]}
[]
422113
{"centers":[]}
[]
422201
{"centers":[]}
[]
422202
{"centers":[]}
[]
422203
{"centers":[{"center_id":58015,"name":"Sc Girnare Tal Nashik","address":"Girnare SC","state_name":"Maharashtra","district_name":"Nashik","block_name":"Nashik","pincode":422203,"lat":20,"long":73,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"ae6a1425-c72d-4f46-a4e1-141c867b2f9e","date":"14-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"],"available_capacity_dose1":0,"available_capacity_dose2":0}]}]}
[{'address': 'Girnare SC',
  'block_name': 'Nashik',
  'center_id': 58015,
  'district_name': 'Nashik',
  'fee_type': 'Free',
  'from': '09:00:00',
  'lat': 20,
  'long': 73,
  'name': 'Sc Girnare Tal Nashik',
  'pincode': 422203,
  'sessions': [{'available_capacity': 0,
                'available_ca

422209
{"centers":[]}
[]
422210
{"centers":[]}
[]
422211
{"centers":[{"center_id":55585,"name":"Barhe PHC","address":"Barhe PHC","state_name":"Maharashtra","district_name":"Nashik","block_name":"Surgana","pincode":422211,"lat":20,"long":73,"from":"09:00:00","to":"17:00:00","fee_type":"Free","sessions":[{"session_id":"a5d6c0d7-8910-452f-95b4-66efa7f80188","date":"14-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"],"available_capacity_dose1":0,"available_capacity_dose2":0},{"session_id":"145a2f1d-21ae-4f37-bae7-c84c21194c95","date":"15-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["09:00AM-11:00AM","11:00AM-01:00PM","01:00PM-03:00PM","03:00PM-05:00PM"],"available_capacity_dose1":0,"available_capacity_dose2":0},{"session_id":"7c529754-fcd9-44be-9553-376eb0786ee0","date":"16-06-2021","available_capacity":0,"min_age_limit":45,"vaccine":"COVISHIELD",

411001
{"centers":[{"center_id":736344,"name":"Usha Nursing Home","address":"Usha Nursing Home 149 M G Road Pune","state_name":"Maharashtra","district_name":"Pune","block_name":"Haveli","pincode":411001,"lat":18,"long":73,"from":"12:30:00","to":"16:30:00","fee_type":"Paid","sessions":[{"session_id":"c25e7338-c988-40e9-a025-e91d8af3f3f4","date":"14-06-2021","available_capacity":0,"min_age_limit":18,"vaccine":"COVISHIELD","slots":["12:30PM-01:30PM","01:30PM-02:30PM","02:30PM-03:30PM","03:30PM-04:30PM"],"available_capacity_dose1":0,"available_capacity_dose2":0},{"session_id":"8864b43d-c3b4-4d0f-9aa2-14dfb5c6c50e","date":"14-06-2021","available_capacity":5,"min_age_limit":45,"vaccine":"COVISHIELD","slots":["12:30PM-01:30PM","01:30PM-02:30PM","02:30PM-03:30PM","03:30PM-04:30PM"],"available_capacity_dose1":0,"available_capacity_dose2":5}],"vaccine_fees":[{"vaccine":"COVISHIELD","fee":"780"}]}]}
[{'address': 'Usha Nursing Home 149 M G Road Pune',
  'block_name': 'Haveli',
  'center_id': 73634

411005
{"centers":[]}
[]
